In [13]:
import pandas as pd
import pyarrow.parquet as pq
import s3fs
s3 = s3fs.S3FileSystem()

In [14]:
import numpy as np
from IPython.display import display
import ast
from collections import Counter

0 for 'human'
1 for 'NLTK_synonym_replacement'
2 for 'chatgpt'
3 for 'summarized'

In [67]:
class_dict = {"human":0,"NLTK_synonym_replacement":1,"chatgpt":2,"summarized":3}

In [3]:
train_df = pq.ParquetDataset('s3://dagpapsubmission/data/data_train_data.parquet', filesystem=s3).read_pandas().to_pandas()

In [15]:
dev_df = pq.ParquetDataset('s3://dagpapsubmission/data/data_dev_data.parquet', filesystem=s3).read_pandas().to_pandas()
dev_df.head()

,text,tokens
index,,
12313,Phylogenetic networks are a generalization of ...,"b'[""Phylogenetic"",""networks"",""are"",""a"",""genera..."
3172,Prediction modelling is more closely aligned w...,"b'[""Prediction"",""modelling"",""is"",""more"",""close..."
6451,The heat transfer exhibits the flow of heat (t...,"b'[""The"",""heat"",""transfer"",""exhibits"",""the"",""f..."
4351,a common experience during superficial ultraso...,"b'[""a"",""common"",""experience"",""during"",""superfi..."
22694,Code metadata Current code version v1.5.9 Perm...,"b'[""Code"",""metadata"",""Current"",""code"",""version..."


In [6]:
train_df["tokens_label_lst"] = train_df.token_label_ids.map(lambda x:ast.literal_eval(x.decode()))

In [7]:
testing = train_df.head()

In [12]:
testing[['tokens_label_lst']].map(lambda x: Counter(x))

,tokens_label_lst
index,
15096,"{0: 2819, 1: 547, 3: 455}"
14428,"{1: 625, 0: 3705}"
2144,"{2: 477, 0: 2135, 1: 827}"
5826,"{0: 2957, 2: 494, 3: 368}"
1452,"{0: 4533, 3: 815, 1: 704, 2: 352}"


In [69]:
train_df.text.nunique()

5000

In [70]:
train_df_transformed = train_df.copy()

In [71]:
train_df_transformed.head()

,text,annotations,tokens,token_label_ids
index,,,,
15096,"Across the world, Emergency Departments are fa...","b'[[0,3779,""human""],[3780,7601,""NLTK_synonym_r...","b'[""Across"",""the"",""world,"",""Emergency"",""Depart...","b'[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0..."
14428,lung Crab is the in the lead make of cancer-re...,"b'[[0,4166,""NLTK_synonym_replacement""],[4167,2...","b'[""lung"",""Crab"",""is"",""the"",""in"",""the"",""lead"",...","b'[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1..."
2144,"The number of osteoporotic fractures, particul...","b'[[0,3264,""chatgpt""],[3265,17179,""human""],[17...","b'[""The"",""number"",""of"",""osteoporotic"",""fractur...","b'[2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2..."
5826,The COVID-19 pandemic has spread to every coun...,"b'[[0,3666,""human""],[3667,6954,""chatgpt""],[695...","b'[""The"",""COVID-19"",""pandemic"",""has"",""spread"",...","b'[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0..."
1452,Endophytic fungi live a significant part of th...,"b'[[0,10489,""human""],[10490,12000,""summarized""...","b'[""Endophytic"",""fungi"",""live"",""a"",""significan...","b'[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0..."


In [72]:
import ast
train_df_transformed["annotation_lst"] = train_df_transformed.annotations.map(lambda x:ast.literal_eval(x.decode()))

In [73]:
train_df_transformed["tokens_label_lst"] = train_df_transformed.token_label_ids.map(lambda x:ast.literal_eval(x.decode()))

In [32]:
train_df_transformed["tokens_lst"] = train_df_transformed.tokens.map(lambda x:ast.literal_eval(x.decode()))

In [33]:
def bytes_to_lst(row):
    annotation = ast.literal_eval(row["annotations"].decode())
    tokens_label = ast.literal_eval(row["token_label_ids"].decode())
    tokens = ast.literal_eval(row["tokens"].decode())
    return annotation,tokens_label,tokens


In [34]:
train_df_transformed.drop(columns=["annotation_lst","tokens_label_lst","tokens_lst"],inplace=True)


In [ ]:
train_df_transformed[["annotation","token_labels","token_lst"]] = train_df_transformed

In [36]:
train_df_transformed.head()

,annotations,tokens,token_label_ids
index,,,
15096,"b'[[0,3779,""human""],[3780,7601,""NLTK_synonym_r...","b'[""Across"",""the"",""world,"",""Emergency"",""Depart...","b'[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0..."
14428,"b'[[0,4166,""NLTK_synonym_replacement""],[4167,2...","b'[""lung"",""Crab"",""is"",""the"",""in"",""the"",""lead"",...","b'[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1..."
2144,"b'[[0,3264,""chatgpt""],[3265,17179,""human""],[17...","b'[""The"",""number"",""of"",""osteoporotic"",""fractur...","b'[2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2..."
5826,"b'[[0,3666,""human""],[3667,6954,""chatgpt""],[695...","b'[""The"",""COVID-19"",""pandemic"",""has"",""spread"",...","b'[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0..."
1452,"b'[[0,10489,""human""],[10490,12000,""summarized""...","b'[""Endophytic"",""fungi"",""live"",""a"",""significan...","b'[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0..."


In [74]:
labels = ast.literal_eval(train_df_transformed.token_label_ids.tolist()[0].decode())
tokens = ast.literal_eval(train_df_transformed.tokens.tolist()[0].decode())
text = train_df_transformed.text.tolist()[0].split(' ')
tokens_not_in_text = [tok for tok in tokens if tok not in text]
text_not_in_tokens = [t for t in text if t not in tokens]
summ_dict = {"Total Text tokens":len(text), 
             "Total unique Text tokens": len(set(text)), 
             "Unqiue Text Tokens not in Token Vocab": len(set(text_not_in_tokens)),
             "Unique tokens in vocab not in text tokens": len(set(tokens_not_in_text))}
print(f"Summary: {summ_dict}")

Summary: {'Total Text tokens': 3822, 'Total unique Text tokens': 1265, 'Unqiue Text Tokens not in Token Vocab': 1, 'Unique tokens in vocab not in text tokens': 0}


In [62]:
text_str_human = train_df_transformed.text.tolist()[0][0:3779]
text_str_nltk = train_df_transformed.text.tolist()[0][3780:7601]

In [63]:
text_str_human

'Across the world, Emergency Departments are facing increasing challenges in delivering high quality patient care due to growing patient numbers and limited hospital resources. Excessive patient waiting time, slow investigation, and delays in making disposition decisions are some causes of overcrowding in Emergency Departments . One way to address the issue of overcrowding is to closely examine turnaround times for all diagnostic tests. A reduction of diagnostic testing time by having only one sample tube for chemistry, hematology and coagulation measurements and an instrument capable of analyzing all these analytes simultaneously can help reduce the time of triaging and or diagnosing by health care professionals. We evaluated a new antithrombotic formulation to measure biochemical, hematological, and d -dimer measurements from a single blood collection tube. Therefore, to analyze hematological, biochemical, and d -dimer in the same blood sample, it is necessary to develop a novel form

In [64]:
text_str_nltk

"In addition, GAO et Camellia State. reported a refreshing mechanics of platelet activating via GPIIb/IIIa mediated outside-in signaling by Lipo Hepin. Due to this blood platelet activating Liquaemin is not suited for haematological depth psychology. various option decoagulant conceptualization have been key such as hirudin and atomic number 12 sulfate. The most victimized assess for the decoagulant action of hirudin are the thrombin clip and the aPTT. Van Resburg et Al. read thrombocyte functionality by light transmitting aggregometry to judge the commercially useable citrate and hirudinized stock solicitation vacuum tube by victimisation unlike agonist. The study dissolve that the commercially usable hirudin tube do not provide a honorable anticoagulant core liken to the commercially useable Na citrate vacuum tube for thrombocyte functionality work. However, there is a clock time restriction of Little Joe h consort with the use of atomic number 11 citrate electron tube. Magnesium sul

In [ ]:
%%time
# For running locally
train_df = pd.read_parquet('/Users/gayatri/Downloads/train_data.parquet', engine="fastparquet")

In [ ]:
print(train_df.shape)
train_df.dtypes

In [20]:
print(type(train_df.iloc[0]['tokens']))
print(type(train_df.iloc[0]['token_label_ids']))
print(type(train_df.iloc[0]['annotations']))

<class 'list'>
<class 'list'>
<class 'list'>


In [47]:
# Verify if len(tokens) == len(token_label_ids) in each row
train_df['len_tokens'] = train_df['tokens'].map(len)
train_df['len_token_ids'] = train_df['token_label_ids'].map(len)
train_df.head()

,text,annotations,tokens,token_label_ids,len_tokens,len_token_ids
index,,,,,,
15096,"Across the world, Emergency Departments are fa...","[[0, 3779, human], [3780, 7601, NLTK_synonym_r...","[Across, the, world,, Emergency, Departments, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",3821,3821
14428,lung Crab is the in the lead make of cancer-re...,"[[0, 4166, NLTK_synonym_replacement], [4167, 2...","[lung, Crab, is, the, in, the, lead, make, of,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",4330,4330
2144,"The number of osteoporotic fractures, particul...","[[0, 3264, chatgpt], [3265, 17179, human], [17...","[The, number, of, osteoporotic, fractures,, pa...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",3439,3439
5826,The COVID-19 pandemic has spread to every coun...,"[[0, 3666, human], [3667, 6954, chatgpt], [695...","[The, COVID-19, pandemic, has, spread, to, eve...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",3819,3819
1452,Endophytic fungi live a significant part of th...,"[[0, 10489, human], [10490, 12000, summarized]...","[Endophytic, fungi, live, a, significant, part...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",6404,6404


In [48]:
train_df[train_df['len_tokens'] != train_df['len_token_ids']]

,text,annotations,tokens,token_label_ids,len_tokens,len_token_ids
index,,,,,,


#### Verified that len(tokens) = len(token_label_ids) in each row

#### Splitting each row of data into multiple rows based on the annotations

In [66]:
def split_text_rows_on_annotation(input_df):
    transformed_rows = []
    for index, row in input_df.iterrows():
        # print(index)
        len_token = len(row['tokens'])
        for new_annot in row['annotations']:
            row_dict = {}
            row_dict['doc_id'] = index
            # row_dict['doc_text'] = row['text']
            row_dict['tokens'] = row['tokens'][new_annot[0]:new_annot[1]]
            row_dict['token_label_ids'] = row['token_label_ids'][new_annot[0]:new_annot[1]]
            row_dict['annotation'] = new_annot[2]
            row_dict['start_id'] = new_annot[0]
            row_dict['end_id'] = new_annot[1]
            row_dict['token_length'] = len(row_dict['tokens'])
            if row_dict['token_length'] == 0:
                row_dict['tokens'] = None
            row_dict['token_label_id_length'] = len(row_dict['token_label_ids'])
            if row_dict['token_label_id_length'] == 0:
                row_dict['token_label_ids'] = None
            
            row_dict['exp_token_length'] = new_annot[1] - new_annot[0]
            row_dict['doc_token_length'] = len_token
            
            if row_dict['tokens'] is not None:
                unique_ids = set(row_dict['token_label_ids'])
                row_dict['len_unique_token_ids'] = len(unique_ids)
                if row_dict['len_unique_token_ids'] == 1:
                    row_dict['unique_token_id'] = unique_ids.pop()
                else:
                    row_dict['unique_token_id'] = unique_ids
            else:
                row_dict['unique_token_id'] = None
                row_dict['len_unique_token_ids'] = 0

            if row_dict['annotation'] == "human":
                row_dict['expected_token_label_id'] = 0
            elif row_dict['annotation'] == "NLTK_synonym_replacement":
                row_dict['expected_token_label_id'] = 1
            elif row_dict['annotation'] == "chatgpt":
                row_dict['expected_token_label_id'] = 2
            elif row_dict['annotation'] == "summarized":
                row_dict['expected_token_label_id'] = 3
            else:
                row_dict['expected_token_label_id'] = None                
                 
            transformed_rows.append(row_dict)
            
    return pd.DataFrame(transformed_rows)

In [85]:
def sanity_checks(transformed_df):
    print("Total rows in the df", transformed_df.shape)
    print()
    # Find the number of Nulls
    print("Null check")
    display(transformed_df.isna().sum().reset_index().rename(columns={'index':'column_names', 0:'null_count'}))
    print()
    
    # Number of rows with token_length != token_label_id_length
    print("Mismatches where tokens and token_label_ids in input file do not have the same size")
    display(transformed_df[transformed_df['token_length'] != transformed_df['token_label_id_length']])
    print()
    
    # Find the number of rows with len_unique_token_ids > 1
    print("Number of rows with more than one unique token id", transformed_df[transformed_df['len_unique_token_ids'] > 1].shape[0])
    print()
    print("Number of rows with only one unique token id", transformed_df[transformed_df['len_unique_token_ids'] == 1].shape[0])
    
    # Number of rows with unique_token_id != expected_token_label_id
    print("Rows with only one unique token id but the token id does not match with the expected token label id")
    display(transformed_df[(transformed_df['len_unique_token_ids'] == 1) & 
            (transformed_df['unique_token_id'] != transformed_df['expected_token_label_id'])])
    print()

    # Label distribution for the data with unique labels
    print("Label distribution for the data with unique labels")
    display(transformed_df[transformed_df['len_unique_token_ids'] == 1][['annotation', 'expected_token_label_id']].value_counts().reset_index())
    print()
    
    # Label distribution for all data
    print("Label distribution for all the data")
    display(transformed_df[['annotation', 'expected_token_label_id']].value_counts().reset_index())
    print()

    print("Token id distribution for unique labels")
    display(transformed_df[transformed_df['len_unique_token_ids'] == 1][['annotation', 'unique_token_id']].value_counts().reset_index())
    print()


In [107]:
# Find the fractions of token length in each category - find actual fraction and expected fraction
def find_fractional_length(df):
    no_null_df = df.dropna(subset=['tokens'], ignore_index=True)
    no_null_stats = no_null_df.groupby(['doc_id', 'annotation', 'expected_token_label_id', 'doc_token_length'
                                       ])[['token_label_id_length', 'exp_token_length']].sum().reset_index()
    max_end_id = no_null_df.groupby(['doc_id', 'annotation', 'expected_token_label_id', 'doc_token_length'
                                       ])[['end_id']].max().reset_index()
    no_null_stats = no_null_stats.merge(max_end_id, how='inner', on=['doc_id', 'annotation', 'expected_token_label_id', 
                                                                     'doc_token_length'])
    
    no_null_stats['exp_doc_length'] = no_null_stats['end_id'] + 1
    no_null_stats['token_length_frac'] = no_null_stats['token_label_id_length']/ no_null_stats['doc_token_length']
    no_null_stats['exp_token_length_frac'] = no_null_stats['exp_doc_length']/ no_null_stats['exp_token_length']

    return no_null_stats

In [108]:
# Pivot the fractional information
def pivot_data_by_annotations(df):
    pivot_df = df.pivot(index=['doc_id'], columns=['annotation'], values=['token_length_frac', 'exp_token_length_frac']).reset_index()
    print("pivot_df.shape", pivot_df.shape)
    display(pivot_df.isna().sum().reset_index().rename(columns={'index':'column_names', 0:'null_count'}))
    pivot_df.fillna(0, inplace=True)
    print(f"\\n After replacing NA with 0s")
    display(pivot_df.isna().sum().reset_index().rename(columns={'index':'column_names', 0:'null_count'}))

    return pivot_df
    

### Main eda/ pipeline

1. Read the train data file
2. Transform the data
3. Perform sanity checks
4. Save the file locally for further feature transformation
5. Find the fractions of token length in each category - find actual fraction and expected fraction
6. Pivot the fractional information

In [67]:
%%time
# Read the train data file

# Transform the data
transformed_train_df = split_text_rows_on_annotation(train_df)
print(transformed_train_df.shape)
transformed_train_df.head()

(26414, 13)
CPU times: user 907 ms, sys: 1.19 s, total: 2.1 s
Wall time: 5.49 s


,doc_id,tokens,token_label_ids,annotation,start_id,end_id,token_length,token_label_id_length,exp_token_length,doc_token_length,len_unique_token_ids,unique_token_id,expected_token_label_id
0,15096,"[Across, the, world,, Emergency, Departments, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",human,0,3779,3779,3779,3779,3821,3,"{0, 1, 3}",0
1,15096,"[Resources,, Project, administration,, Supervi...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NLTK_synonym_replacement,3780,7601,41,41,3821,3821,1,0,1
2,15096,None,None,summarized,7602,9401,0,0,1799,3821,0,None,3
3,15096,None,None,human,9402,25014,0,0,15612,3821,0,None,0
4,14428,"[lung, Crab, is, the, in, the, lead, make, of,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",NLTK_synonym_replacement,0,4166,4166,4166,4166,4330,2,"{0, 1}",1


In [86]:
# Perform sanity checks
sanity_checks(transformed_train_df)

Total rows in the df (26414, 13)

Null check


,column_names,null_count
0,doc_id,0
1,tokens,18630
2,token_label_ids,18630
3,annotation,0
4,start_id,0
5,end_id,0
6,token_length,0
7,token_label_id_length,0
8,exp_token_length,0
9,doc_token_length,0



Mismatches where tokens and token_label_ids in input file do not have the same size


,doc_id,tokens,token_label_ids,annotation,start_id,end_id,token_length,token_label_id_length,exp_token_length,doc_token_length,len_unique_token_ids,unique_token_id,expected_token_label_id



Number of rows with more than one unique token id 6588

Number of rows with only one unique token id 1196
Rows with only one unique token id but the token id does not match with the expected token label id


...

,annotation,expected_token_label_id,count
0,human,0,663
1,summarized,3,192
2,chatgpt,2,191
3,NLTK_synonym_replacement,1,150



Label distribution for all the data


,annotation,expected_token_label_id,count
0,human,0,13346
1,chatgpt,2,4447
2,summarized,3,4376
3,NLTK_synonym_replacement,1,4245



Token id distribution for unique labels


,annotation,unique_token_id,count
0,human,0,645
1,chatgpt,0,184
2,summarized,0,184
3,NLTK_synonym_replacement,0,142
4,human,1,9
5,chatgpt,1,5
6,human,2,5
7,NLTK_synonym_replacement,1,4
8,NLTK_synonym_replacement,3,4
9,human,3,4


In [88]:
%%time
# Save the file locally for further feature transformation
transformed_train_df.to_csv('../data/tranformed_train_data.csv', index=False)

In [109]:
# Find the fractions of token length in each category - find actual fraction and expected fraction
fractions = find_fractional_length(transformed_train_df)
print(fractions.shape)
fractions.head()

(7405, 10)


,doc_id,annotation,expected_token_label_id,doc_token_length,token_label_id_length,exp_token_length,end_id,exp_doc_length,token_length_frac,exp_token_length_frac
0,10,human,0,6604,4979,8292,9917,9918,0.753937,1.196093
1,10,summarized,3,6604,1624,1624,1624,1625,0.245912,1.000616
2,13,human,0,6099,6099,19153,19153,19154,1.000000,1.000052
3,19,human,0,3118,3118,7747,7747,7748,1.000000,1.000129
4,28,human,0,3847,2249,3503,5101,5102,0.584611,1.456466


In [110]:
# Pivot the fractional information
pivot_df = pivot_data_by_annotations(fractions)
pivot_df.head()

pivot_df.shape (5000, 9)


,level_0,annotation,null_count
0,doc_id,,0
1,token_length_frac,NLTK_synonym_replacement,4145
2,token_length_frac,chatgpt,4092
3,token_length_frac,human,225
4,token_length_frac,summarized,4133
5,exp_token_length_frac,NLTK_synonym_replacement,4145
6,exp_token_length_frac,chatgpt,4092
7,exp_token_length_frac,human,225
8,exp_token_length_frac,summarized,4133


\n After replacing NA with 0s


,level_0,annotation,null_count
0,doc_id,,0
1,token_length_frac,NLTK_synonym_replacement,0
2,token_length_frac,chatgpt,0
3,token_length_frac,human,0
4,token_length_frac,summarized,0
5,exp_token_length_frac,NLTK_synonym_replacement,0
6,exp_token_length_frac,chatgpt,0
7,exp_token_length_frac,human,0
8,exp_token_length_frac,summarized,0


doc_id        token_length_frac                               \
annotation        NLTK_synonym_replacement chatgpt     human summarized   
0              10                      0.0     0.0  0.753937   0.245912   
1              13                      0.0     0.0  1.000000   0.000000   
2              19                      0.0     0.0  1.000000   0.000000   
3              28                      0.0     0.0  0.584611   0.415129   
4              40                      0.0     0.0  1.000000   0.000000   

              exp_token_length_frac                               
annotation NLTK_synonym_replacement chatgpt     human summarized  
0                               0.0     0.0  1.196093   1.000616  
1                               0.0     0.0  1.000052   0.000000  
2                               0.0     0.0  1.000129   0.000000  
3                               0.0     0.0  1.456466   1.000626  
4                               0.0     0.0  1.000040   0.000000

### Scratch pad

In [89]:
no_null_data = transformed_train_df.dropna(subset=['tokens'], ignore_index=True)
print(no_null_data.shape)
no_null_data.head()

(7784, 13)


,doc_id,tokens,token_label_ids,annotation,start_id,end_id,token_length,token_label_id_length,exp_token_length,doc_token_length,len_unique_token_ids,unique_token_id,expected_token_label_id
0,15096,"[Across, the, world,, Emergency, Departments, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",human,0,3779,3779,3779,3779,3821,3,"{0, 1, 3}",0
1,15096,"[Resources,, Project, administration,, Supervi...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NLTK_synonym_replacement,3780,7601,41,41,3821,3821,1,0,1
2,14428,"[lung, Crab, is, the, in, the, lead, make, of,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",NLTK_synonym_replacement,0,4166,4166,4166,4166,4330,2,"{0, 1}",1
3,14428,"[after, surgery), is, of, great, importance, f...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",human,4167,27851,163,163,23684,4330,1,0,0
4,2144,"[The, number, of, osteoporotic, fractures,, pa...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",chatgpt,0,3264,3264,3264,3264,3439,3,"{0, 1, 2}",2


In [92]:
no_null_data.groupby(['doc_id', 'annotation', 'expected_token_label_id', 'doc_token_length', 'end_id'])[['token_label_id_length', 'exp_token_length']].sum().reset_index()

,doc_id,annotation,expected_token_label_id,doc_token_length,end_id,token_label_id_length,exp_token_length
0,10,human,0,6604,9917,4979,8292
1,10,summarized,3,6604,1624,1624,1624
2,13,human,0,6099,19153,6099,19153
3,19,human,0,3118,7747,3118,7747
4,28,human,0,3847,5101,2249,3503
...,...,...,...,...,...,...,...
7779,24984,human,0,6913,8416,1796,3299
7780,24984,summarized,3,6913,5116,1254,1254
7781,24991,human,0,9803,50049,9803,50049
7782,24993,chatgpt,2,6241,2783,2783,2783


In [103]:
fractrions = find_fractional_length(no_null_data)
print(fractrions.shape)
fractrions.head()

(7405, 10)


,doc_id,annotation,expected_token_label_id,doc_token_length,token_label_id_length,exp_token_length,end_id,exp_doc_length,token_length_frac,exp_token_length_frac
0,10,human,0,6604,4979,8292,9917,9918,0.753937,1.196093
1,10,summarized,3,6604,1624,1624,1624,1625,0.245912,1.000616
2,13,human,0,6099,6099,19153,19153,19154,1.000000,1.000052
3,19,human,0,3118,3118,7747,7747,7748,1.000000,1.000129
4,28,human,0,3847,2249,3503,5101,5102,0.584611,1.456466


In [106]:
fractrions.pivot(index=['doc_id'], columns=['annotation'], values=['token_length_frac', 'exp_token_length_frac']).reset_index()

doc_id        token_length_frac                                 \
annotation        NLTK_synonym_replacement   chatgpt     human summarized   
0              10                      NaN       NaN  0.753937   0.245912   
1              13                      NaN       NaN  1.000000        NaN   
2              19                      NaN       NaN  1.000000        NaN   
3              28                      NaN       NaN  0.584611   0.415129   
4              40                      NaN       NaN  1.000000        NaN   
...           ...                      ...       ...       ...        ...   
4995        24975                      NaN       NaN  1.000000        NaN   
4996        24977                      NaN  0.509058  0.490791        NaN   
4997        24984                      NaN       NaN  0.818313   0.181397   
4998        24991                      NaN       NaN  1.000000        NaN   
4999        24993                      NaN  0.445922  0.553918        NaN   

              exp_token_length_frac                                 
annotation NLTK_synonym_replacement   chatgpt     human summarized  
0                               NaN       NaN  1.196093   1.000616  
1                               NaN       NaN  1.000052        NaN  
2                               NaN       NaN  1.000129        NaN  
3                               NaN       NaN  1.456466   1.000626  
4                               NaN       NaN  1.000040        NaN  
...                             ...       ...       ...        ...  
4995                            NaN       NaN  1.000253        NaN  
4996                            NaN  1.925726  1.000308        NaN  
4997                            NaN       NaN  1.175559   4.080542  
4998                            NaN       NaN  1.000020        NaN  
4999                            NaN  1.000359  1.427213        NaN  

[5000 rows x 9 columns]

In [101]:
fractrions.groupby(['doc_id', 'annotation'])['doc_id'].value_counts().reset_index()

doc_id  annotation
10      human         1
        summarized    1
13      human         1
19      human         1
28      human         1
                     ..
24984   human         2
        summarized    1
24991   human         1
24993   chatgpt       1
        human         1
Name: count, Length: 7405, dtype: int64